# Data Gathering Part 2
Download images and make labels

In [1]:
import requests, re, time
import torch, torchvision
from torch import nn, optim
from torchvision import datasets, models, transforms
import matplotlib.pyplot as pl
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import os, glob
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## NO names or dimensions

In [ ]:
df = pd.read_csv('/content/drive/My Drive/CPEN 291/project/artsy.csv')
df.head()

,Unnamed: 0,image,price
0,0,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,"$12,000"
1,1,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,"$6,250"
2,2,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,"$5,000"
3,3,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,$475
4,4,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,"$5,100"


In [ ]:
# Currency convertor/ Price formatter
# Assuming that exchange rates have not changed drastically since the time of auction
import re
def cr_cnvrtr(oldPrice):
  rates = {
      "$" : 1,
      "£" : 1.37,
      "PHP " : 0.02,
      "PHP\xa0" : 0.02,
      "CZK " : 0.04,
      "CZK\xa0" : 0.04,
      "CHF " : 1.06,
      "CHF\xa0" : 1.06,
      "€" : 1.17,
      "CA$" : 0.79,
      "A$" : 0.76,
      "PLN " : 0.25,
      "PLN\xa0" : 0.25,
      "NZ$" : 0.7,
      "ZAR " : 0.07,
      "ZAR\xa0" : 0.07,
      "NT$" : 0.04
           }
  if ('$' in oldPrice):
    oldPlist = re.split('1|2|3|4|5|6|7|8|9', oldPrice);
    newP = float(oldPrice.replace(oldPlist[0], '').replace(',', '')) * rates[oldPlist[0]]
  else:
    newP = float(oldPrice.replace(',', ''))
  #print(newP)
  return newP

In [ ]:
df["price"] = df["price"].apply(cr_cnvrtr)
df.head()

,Unnamed: 0,image,price
0,0,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,12000.0
1,1,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,6250.0
2,2,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,5000.0
3,3,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,475.0
4,4,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,5100.0


In [ ]:
labels = pd.DataFrame(columns=['name', 'price'])
labels.head()


,name,price


In [ ]:
# liveauctioneers
for index, row in df.iterrows():
  if (row['price'] <= 20000): # Upper threshold we chose
    newRow = {'name': f'liveauctioneer_{index}.jpg', 'price': row['price']};
    resp = requests.get(row['image'])
    with open(f"/content/drive/My Drive/CPEN 291/project/data/liveauctioneer_{index}.jpg", 'wb') as f:
        f.write(resp.content)
    labels = labels.append(newRow, ignore_index=True)
    time.sleep(5)

labels.to_csv('/content/drive/My Drive/CPEN 291/project/data/liveauctioneer_labels.csv')


In [ ]:
# artsy
for index, row in df.iterrows():
  if (row['price'] <= 20000): # Upper threshold we chose
    newRow = {'name': f'artsy_{index}.jpg', 'price': row['price']};
    resp = requests.get(row['image'])
    with open(f"/content/drive/My Drive/CPEN 291/project/data/artsy_{index}.jpg", 'wb') as f:
        f.write(resp.content)
    labels = labels.append(newRow, ignore_index=True)
    time.sleep(2)

labels.to_csv('/content/drive/My Drive/CPEN 291/project/data/artsy_labels.csv')

In [ ]:
labels = pd.DataFrame(columns=['name', 'price'])
labels.head()

,name,price


In [ ]:
# christies
df = pd.read_excel('/content/drive/My Drive/CPEN 291/project/christies.xlsx')
df.head()

,Unnamed: 0,image,Unnamed: 2,price
0,1,https://www.christies.com/img/LotImages/2021/C...,12000,16800
1,2,https://www.christies.com/img/LotImages/2021/C...,6000,8400
2,3,https://www.christies.com/img/LotImages/2021/C...,10000,14000
3,4,https://www.christies.com/img/LotImages/2021/C...,5000,7000
4,5,https://www.christies.com/img/LotImages/2021/C...,2500,3500


In [ ]:
# christies
for index, row in df.iterrows():
  if (row['price'] <= 20000): # Upper threshold we chose
    newRow = {'name': f'christies_{index}.jpg', 'price': row['price']};
    resp = requests.get(row['image'])
    with open(f"/content/drive/My Drive/CPEN 291/project/data/christies_{index}.jpg", 'wb') as f:
        f.write(resp.content)
    labels = labels.append(newRow, ignore_index=True)
    time.sleep(5)

labels.to_csv('/content/drive/My Drive/CPEN 291/project/data/christies_labels.csv')


In [ ]:
# artsper
df = pd.read_excel('/content/drive/My Drive/CPEN 291/project/artsper.xlsx')
print(df.head())
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
  if (row['price'] <= 20000): # Upper threshold we chose
    newRow = {'name': f'artsper_{index}.jpg', 'price': row['price']};
    resp = requests.get(row['image'])
    with open(f"/content/drive/My Drive/CPEN 291/project/data/artsper_{index}.jpg", 'wb') as f:
        f.write(resp.content)
    labels = labels.append(newRow, ignore_index=True)
    time.sleep(2)

labels.to_csv('/content/drive/My Drive/CPEN 291/project/data/artsper_labels.csv')

   Unnamed: 0                                              image  price
0           1  https://media.artsper.com/artwork/1042986_1_gr...    307
1           2  https://media.artsper.com/artwork/1057140_1_gr...   3440
2           3  https://media.artsper.com/artwork/1053681_1_gr...   3071
3           4  https://media.artsper.com/artwork/1060317_1_gr...   2076
4           5  https://media.artsper.com/artwork/1061736_1_gr...    491


In [ ]:
a = labels.loc[labels['name'] == 'liveauctioneer_101.jpg'].index[0]
labels.loc[labels['name'] == 'liveauctioneer_101.jpg']['price'][a]

In [ ]:
# combine all labels csv files
chr = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data/christies_labels.csv')
live = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data/liveauctioneer_labels.csv')
artsy = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data/artsy_labels.csv')
artsper = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data/artsper_labels.csv')

frames = [chr, live, artsy, artsper]
result = pd.concat(frames)

result.to_csv('/content/drive/My Drive/CPEN 291/project/data/all_labels.csv')


## Paintings with artists' names and dimensions

In [ ]:
labels = pd.DataFrame(columns=['imageName', 'price', 'artist', 'width', 'height'])
labels_old = pd.DataFrame(columns=['name', 'price'])
# gallerytoday
df = pd.read_excel('/content/drive/My Drive/CPEN 291/project/datagathering/gallerytoday.xlsx')
print(df.head())
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
  dimensions = row['dimensions (inch)']
  dimensions = dimensions.replace('"', '')
  dimensions = dimensions.replace(' ', '')
  wh = dimensions.split('x')
  newRow = {'imageName': f'gallerytoday_{index}.jpg', 'price': row['price(USD)'], 'artist': row['artists'], 'width': wh[0], 'height': wh[1]};
  newRow_old = {'name': f'gallerytoday_{index}.jpg', 'price': row['price(USD)']};

  labels = labels.append(newRow, ignore_index=True)
  labels_old = labels_old.append(newRow_old, ignore_index=True)

  if (os.path.exists(f'/content/drive/My Drive/CPEN 291/project/data/gallerytoday_{index}.jpg')):
    continue
  resp = requests.get(row['image'])
  with open(f"/content/drive/My Drive/CPEN 291/project/data_nd/gallerytoday_{index}.jpg", 'wb') as f:
      f.write(resp.content)
  with open(f"/content/drive/My Drive/CPEN 291/project/data/gallerytoday_{index}.jpg", 'wb') as fo:
      fo.write(resp.content)
  time.sleep(0.5)

labels.to_csv('/content/drive/My Drive/CPEN 291/project/data_nd/gallerytoday_labels.csv')
labels_old.to_csv('/content/drive/My Drive/CPEN 291/project/data/gallerytoday_labels.csv')

   Unnamed: 0 artists  ... dimensions (inch) price(USD)
0           1  A. Roy  ...         20" x 18"        800
1           2  A. Roy  ...         15" x 18"        500
2           3  A. Roy  ...         20" x 12"        900
3           4  A. Roy  ...         20" x 12"        900
4           5  A. Roy  ...         28" x 21"       1200

[5 rows x 5 columns]


In [ ]:
for filename in glob.glob("/content/drive/My Drive/CPEN 291/project/data/saatchi*"):
    os.remove(filename) 
for filename in glob.glob("/content/drive/My Drive/CPEN 291/project/data_nd/saatchi*"):
    os.remove(filename) 


In [ ]:
labels = pd.DataFrame(columns=['imageName', 'price', 'artist', 'width', 'height'])
labels_old = pd.DataFrame(columns=['name', 'price'])
# saatchi
df = pd.read_excel('/content/drive/My Drive/CPEN 291/project/datagathering/Saatchi.xlsx')
df1 = pd.read_excel('/content/drive/My Drive/CPEN 291/project/datagathering/Saatchi1.xlsx')
index_counter = 0
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
  dimensions = row['Dimension']
  dimensions = dimensions = dimensions.split('H')[0]
  dimensions = dimensions.replace('W', '')
  dimensions = dimensions.replace(' ', '')
  wh = dimensions.split('x')
  price = row['Price']
  price = price.replace('$', '')
  price = price.replace(',', '')
  price = int(price)
  newRow = {'imageName': f'saatchi_{index}.jpg', 'price': price, 'artist': row['Artist'], 'width': wh[0], 'height': wh[1]};
  newRow_old = {'name': f'saatchi_{index}.jpg', 'price': price};

  labels = labels.append(newRow, ignore_index=True)
  labels_old = labels_old.append(newRow_old, ignore_index=True)
  index_counter = index + 1

  if (os.path.exists(f'/content/drive/My Drive/CPEN 291/project/data/saatchi_{index}.jpg')):
    continue
  resp = requests.get(row['Image'])
  with open(f"/content/drive/My Drive/CPEN 291/project/data_nd/saatchi_{index}.jpg", 'wb') as f:
      f.write(resp.content)
  with open(f"/content/drive/My Drive/CPEN 291/project/data/saatchi_{index}.jpg", 'wb') as fo:
      fo.write(resp.content)
  time.sleep(0.1)

for index, row in tqdm(df1.iterrows(), total=df1.shape[0]):
  dimensions = row['Dimension']
  dimensions = dimensions.split('H')[0]
  dimensions.replace('W', '')
  dimensions.replace(' ', '')
  wh = dimensions.split('x')
  price = row['Price']
  if (not (isinstance(price, float) or isinstance(price, int))):
    price = price.replace('$', '')
    price = price.replace(',', '')
    if ('Prints' in price):
      continue
    price = float(price)
  newRow = {'imageName': f'saatchi_{index_counter + index}.jpg', 'price': price, 'artist': row['Artist'], 'width': wh[0], 'height': wh[1]};
  newRow_old = {'name': f'saatchi_{index_counter + index}.jpg', 'price': price};

  labels = labels.append(newRow, ignore_index=True)
  labels_old = labels_old.append(newRow_old, ignore_index=True)

  if (os.path.exists(f'/content/drive/My Drive/CPEN 291/project/data/saatchi_{index_counter + index}.jpg')):
    continue
  resp = requests.get(row['Image'])
  with open(f"/content/drive/My Drive/CPEN 291/project/data_nd/saatchi_{index_counter + index}.jpg", 'wb') as f:
      f.write(resp.content)
  with open(f"/content/drive/My Drive/CPEN 291/project/data/saatchi_{index_counter + index}.jpg", 'wb') as fo:
      fo.write(resp.content)
  time.sleep(0.1)

labels.to_csv('/content/drive/My Drive/CPEN 291/project/data_nd/saatchi_labels.csv')
labels_old.to_csv('/content/drive/My Drive/CPEN 291/project/data/saatchi_labels.csv')

In [4]:
# combine new labels csv files with all_labels
gt = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data_nd/gallerytoday_labels.csv', index_col=0)
saa = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data_nd/saatchi_labels.csv', index_col=0)
am = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data_nd/artmajeur_labels.csv', index_col=0)

frames = [gt, saa, am]
result = pd.concat(frames)

result.to_csv('/content/drive/My Drive/CPEN 291/project/data_nd/all_labels.csv')

In [ ]:
# combine new labels csv files with all_labels
all_labels = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data/all_labels.csv', index_col=0)
gt = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data/gallerytoday_labels.csv', index_col=0)
saa = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data/saatchi_labels.csv', index_col=0)
am = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data/artmajeur_labels.csv', index_col=0)

frames = [all_labels, gt, saa, am]
result = pd.concat(frames)

result.to_csv('/content/drive/My Drive/CPEN 291/project/data/all_labels.csv')

In [ ]:
nd = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data_nd/all_labels.csv')
nd = nd.dropna()
for index, i in nd.iterrows():
  if (math.isnan(i['price'])):
    print(i)
    input()

nd.to_csv('/content/drive/My Drive/CPEN 291/project/data_nd/all_labels.csv')

In [ ]:
sa = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data_nd/saatchi_labels.csv')
for index, i in sa.iterrows():
  if (math.isnan(i['price'])):
    print(i)
    os.remove('/content/drive/My Drive/CPEN 291/project/data_nd/' + i['imageName']) 

#nd.to_csv('/content/drive/My Drive/CPEN 291/project/data_nd/all_labels.csv')

Unnamed: 0                4880
imageName     saatchi_5135.jpg
price                      NaN
artist             Anna McNeil
width                  25.6 W 
height                    31.9
Name: 4880, dtype: object
Unnamed: 0                4881
imageName     saatchi_5136.jpg
price                      NaN
artist             Anna McNeil
width                   8.5 W 
height                     5.7
Name: 4881, dtype: object
Unnamed: 0                4882
imageName     saatchi_5137.jpg
price                      NaN
artist             Anna McNeil
width                    13 W 
height                    16.1
Name: 4882, dtype: object
Unnamed: 0                4883
imageName     saatchi_5138.jpg
price                      NaN
artist             Anna McNeil
width                  44.9 W 
height                    57.5
Name: 4883, dtype: object
Unnamed: 0                4884
imageName     saatchi_5139.jpg
price                      NaN
artist             Anna McNeil
width                  15.7 

In [ ]:
sa = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data/saatchi_labels.csv')
for index, i in sa.iterrows():
  if (math.isnan(i['price'])):
    print(i)
    os.remove('/content/drive/My Drive/CPEN 291/project/data/' + i['name']) 

#nd.to_csv('/content/drive/My Drive/CPEN 291/project/data_nd/all_labels.csv')

Unnamed: 0                4880
name          saatchi_5135.jpg
price                      NaN
Name: 4880, dtype: object
Unnamed: 0                4881
name          saatchi_5136.jpg
price                      NaN
Name: 4881, dtype: object
Unnamed: 0                4882
name          saatchi_5137.jpg
price                      NaN
Name: 4882, dtype: object
Unnamed: 0                4883
name          saatchi_5138.jpg
price                      NaN
Name: 4883, dtype: object
Unnamed: 0                4884
name          saatchi_5139.jpg
price                      NaN
Name: 4884, dtype: object
Unnamed: 0                4885
name          saatchi_5140.jpg
price                      NaN
Name: 4885, dtype: object
Unnamed: 0                4886
name          saatchi_5141.jpg
price                      NaN
Name: 4886, dtype: object
Unnamed: 0                4887
name          saatchi_5142.jpg
price                      NaN
Name: 4887, dtype: object
Unnamed: 0                4888
name          saa

In [ ]:
ama = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data/artmajeur_labels.csv')
for index, i in ama.iterrows():
  if (math.isnan(i['price'])):
    print(i)
    os.remove('/content/drive/My Drive/CPEN 291/project/data/' + i['name']) 

#nd.to_csv('/content/drive/My Drive/CPEN 291/project/data_nd/all_labels.csv')


In [ ]:
d = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data/all_labels.csv')
d = nd.dropna()
for index, i in d.iterrows():
  if (math.isnan(i['price'])):
    print(i)
    input()

d.to_csv('/content/drive/My Drive/CPEN 291/project/data/all_labels.csv')


In [ ]:
labels = pd.DataFrame(columns=['imageName', 'price', 'artist', 'width', 'height'])
labels_old = pd.DataFrame(columns=['name', 'price'])
# artmajeur
df = pd.read_excel('/content/drive/My Drive/CPEN 291/project/datagathering/Artmajeur.xlsx')
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
  if ('safeSearchPlaceholder' in row['image']):
    continue
  newRow = {'imageName': f'artmajeur_{index}.jpg', 'price': row['price (USD)'], 'artist': row['artist'], 'width': row['width (in)'], 'height': row['height (in)']};
  newRow_old = {'name': f'artmajeur_{index}.jpg', 'price': price};

  labels = labels.append(newRow, ignore_index=True)
  labels_old = labels_old.append(newRow_old, ignore_index=True)

  if (os.path.exists(f'/content/drive/My Drive/CPEN 291/project/data/artmajeur_{index}.jpg')):
    continue
  resp = requests.get(row['image'])
  with open(f"/content/drive/My Drive/CPEN 291/project/data_nd/artmajeur_{index}.jpg", 'wb') as f:
      f.write(resp.content)
  with open(f"/content/drive/My Drive/CPEN 291/project/data/artmajeur_{index}.jpg", 'wb') as fo:
      fo.write(resp.content)
  time.sleep(0.1)


labels.to_csv('/content/drive/My Drive/CPEN 291/project/data_nd/artmajeur_labels.csv')
labels_old.to_csv('/content/drive/My Drive/CPEN 291/project/data/artmajeur_labels.csv')

In [ ]:
# combine artmajeur labels csv files with all_labels
all_labels = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data_nd/all_labels.csv')
am = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data_nd/artmajeur_labels.csv')

frames = [all_labels, am]
result = pd.concat(frames)

result.to_csv('/content/drive/My Drive/CPEN 291/project/data_nd/all_labels.csv')


In [ ]:
# combine artmajeur labels csv files with all_labels
all_labels = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data/all_labels.csv')
am = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data/artmajeur_labels.csv')

frames = [all_labels, am]
result = pd.concat(frames)

result.to_csv('/content/drive/My Drive/CPEN 291/project/data/all_labels.csv')

In [ ]:
old_data_all_labels.head()

,Unnamed: 0,Unnamed: 0.1,imageName,price,artist,width,height
0,0,0,gallerytoday_0.jpg,800.0,A. Roy,20,18.0
1,1,1,gallerytoday_1.jpg,500.0,A. Roy,15,18.0
2,2,2,gallerytoday_2.jpg,900.0,A. Roy,20,12.0
3,3,3,gallerytoday_3.jpg,900.0,A. Roy,20,12.0
4,4,4,gallerytoday_4.jpg,1200.0,A. Roy,28,21.0


In [ ]:
old_data_all_labels = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data/all_labels.csv', index_col=0)
#old_data_all_labels = old_data_all_labels.dropna()
for index, i in old_data_all_labels.iterrows():
  if (not os.path.exists('/content/drive/My Drive/CPEN 291/project/data/' + i['name'])):
    print(i)
    input()
  if (math.isnan(i['price'])):
    print(i)
    input()

input()
old_data_all_labels.to_csv('/content/drive/My Drive/CPEN 291/project/data/all_labels.csv')


In [ ]:
data_all_labels = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data_nd/all_labels.csv')
data_all_labels = data_all_labels.dropna()
for index, i in data_all_labels.iterrows():
  if (not os.path.exists('/content/drive/My Drive/CPEN 291/project/data_nd/' + i['imageName'])):
    print(i)
    input()
  if (math.isnan(i['price'])):
    print(i)
    input()

input()
data_all_labels.to_csv('/content/drive/My Drive/CPEN 291/project/data_nd/all_labels.csv')


lajdhfowuen


In [3]:
old_data_all_labels = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data_classes/all_labels.csv', index_col=0)
old_data_all_labels = old_data_all_labels.loc[:, ~old_data_all_labels.columns.str.contains('^Unnamed')]
old_data_all_labels.head()
old_data_all_labels.to_csv('/content/drive/My Drive/CPEN 291/project/data_classes/all_labels.csv')

In [ ]:
data_all_labels = data_all_labels.loc[:, ~data_all_labels.columns.str.contains('^Unnamed')]
print(data_all_labels.head())
input()
data_all_labels.to_csv('/content/drive/My Drive/CPEN 291/project/data_nd/all_labels.csv')


            imageName   price  artist width  height
0  gallerytoday_0.jpg   800.0  A. Roy    20    18.0
1  gallerytoday_1.jpg   500.0  A. Roy    15    18.0
2  gallerytoday_2.jpg   900.0  A. Roy    20    12.0
3  gallerytoday_3.jpg   900.0  A. Roy    20    12.0
4  gallerytoday_4.jpg  1200.0  A. Roy    28    21.0
yes


In [46]:
data_all_labels = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data_nd/all_labels.csv')
for index in data_all_labels.index:

  if (isinstance(data_all_labels.loc[index, 'width'], str)):
    data_all_labels.loc[index, 'width'] = float(data_all_labels.loc[index, 'width'].replace('W', ''))
  #if (not (isinstance(i['width'], float) or isinstance(i['width'], int))):
  #  i['width'] = float(i['width'].replace('W', ''))

print(data_all_labels.head())
input()
data_all_labels.to_csv('/content/drive/My Drive/CPEN 291/project/data_nd/all_labels.csv')


   Unnamed: 0           imageName   price  artist width  height
0           0  gallerytoday_0.jpg   800.0  A. Roy    20    18.0
1           1  gallerytoday_1.jpg   500.0  A. Roy    15    18.0
2           2  gallerytoday_2.jpg   900.0  A. Roy    20    12.0
3           3  gallerytoday_3.jpg   900.0  A. Roy    20    12.0
4           4  gallerytoday_4.jpg  1200.0  A. Roy    28    21.0
ok


## Artists

In [9]:
df = pd.read_csv('/content/drive/My Drive/CPEN 291/project/data_nd/all_labels.csv')


In [4]:
df.head()

,Unnamed: 0,imageName,price,artist,width,height
0,0,gallerytoday_0.jpg,800.0,A. Roy,20.0,18.0
1,1,gallerytoday_1.jpg,500.0,A. Roy,15.0,18.0
2,2,gallerytoday_2.jpg,900.0,A. Roy,20.0,12.0
3,3,gallerytoday_3.jpg,900.0,A. Roy,20.0,12.0
4,4,gallerytoday_4.jpg,1200.0,A. Roy,28.0,21.0


In [36]:
artists_list = df.artist.unique()

In [39]:
len(artists_list)

5197

In [38]:
artists_list

array(['A. Roy', 'Andres Vivo', 'Alexander Dobrodiy', ...,
       'Янина Руденко', '⭐️Fov⭐️', '志泽 吕'], dtype=object)

In [42]:
np.savetxt('/content/drive/My Drive/CPEN 291/project/artists_list.gz', artists_list, fmt="%s", delimiter=',')

In [44]:
len(np.loadtxt('/content/drive/My Drive/CPEN 291/project/artists_list.gz', str, delimiter=","))

5197

In [33]:
np.save('/content/drive/My Drive/CPEN 291/project/artists_list.npy', artists_list)

In [34]:
artists_list = np.load('/content/drive/My Drive/CPEN 291/project/data_nd/artists_list.npy', allow_pickle=True)

In [22]:
index = np.where(artists_list == '405')[0][0]

IndexError: ignored

In [25]:
for i in artists_list:
  print(i)

In [35]:
len(artists_list)

2015

In [ ]:
artist_list[]